# Streaming receiver

In [ ]:
import sys
import pyzed.sl as sl
import cv2

import math
import numpy as np


In [ ]:
def get_distance(point3D):
    distance = math.sqrt(point3D[0] * point3D[0] + point3D[1] * point3D[1] + point3D[2] * point3D[2])

    if not np.isnan(distance) and not np.isinf(distance):
        distance = round(distance)
        print("Distance to Camera : {0} mm\n".format(distance))
    return distance

In [ ]:
def main():

    init = sl.InitParameters()
    init.camera_resolution = sl.RESOLUTION.RESOLUTION_HD720
    init.depth_mode = sl.DEPTH_MODE.DEPTH_MODE_PERFORMANCE

    #read from streaming
    sys.argv[1] = '127.0.0.1'
    if (len(sys.argv) > 1) :
        ip = sys.argv[1]
        init.set_from_stream(ip)
    else :
        print('Usage : python3 streaming_receiver.py ip')
        exit(1)
        
    #read from SVO file
#     filepath = 'record_1017.svo'
#     print("Reading SVO file: {0}".format(filepath))

    init = sl.InitParameters(svo_input_filename=filepath,svo_real_time_mode=False)

    cam = sl.Camera()
    status = cam.open(init)
    if status != sl.ERROR_CODE.SUCCESS:
        print(repr(status))
        exit(1)

    runtime = sl.RuntimeParameters()
    
    image = sl.Mat() #image
    depth = sl.Mat() #depth map
    depth_for_display = sl.Mat() #depth map,scale its values to [0, 255]
    point_cloud = sl.Mat() #colored point cloud.

    key = ''
    print("  Quit : CTRL+C\n")
    while key != 113:
        err = cam.grab(runtime)
        if (err == sl.ERROR_CODE.SUCCESS) :
            # Retrieve left image
            cam.retrieve_image(image, sl.VIEW.VIEW_LEFT)
            cam.retrieve_image(depth_for_display, sl.VIEW.VIEW_DEPTH)
            # Retrieve depth map. Depth is aligned on the left image
            cam.retrieve_measure(depth, sl.MEASURE.MEASURE_DEPTH)
            # Retrieve colored point cloud. Point cloud is aligned on the left image.
            cam.retrieve_measure(point_cloud, sl.MEASURE.MEASURE_XYZRGBA)
            
#             #test
#             x = round(image.get_width() / 2)
#             y = round(image.get_height() / 2)
#             err, depth_value = depth.get_value(x,y)
#             print("depth_value at ({0}, {1}): {2} mm\n".format(x, y, round(depth_value)))
#             err, point3D = point_cloud.get_value(x,y)
#             get_distance(point3D)
#             #test
            
            cv2.imshow("ZED", image.get_data())
#             cv2.imshow("Depth img", depth_for_display.get_data())
            key = cv2.waitKey(1)
        else :
            key = cv2.waitKey(1)

    cam.close()



In [ ]:
if __name__ == "__main__":
    main()